In [1]:
import requests
import lxml.html as lh
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re

In [490]:
#url='https://www.houstontx.gov/police/news.htm'
url='https://www.houstontx.gov/police/nr/2019/dec/index.htm'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [491]:
soup = BeautifulSoup(page.text, 'lxml')
gdp = soup.find_all("table", attrs={"class": "default"})
print("Number of tables on site: ",len(gdp))


Number of tables on site:  1


In [496]:
table1 = gdp[0]
body = table1.find_all("tr")
# Head values (Column names) are the first items of the body list
head = body[0] # 0th item is the header row
body_rows = body[1:] # All other items becomes the rest of the rows

# Lets now iterate through the head HTML code and make list of clean headings

# Declare empty list to keep Columns names
headings = []
for item in head.find_all("th"): # loop through all th elements
    # convert the th elements to text and strip "\n"
    item = (item.text).rstrip("\n")
    # append the clean column name to headings
    headings.append(item)
print(headings)
# Next is now to loop though the rest of the rows

#print(body_rows[0])
all_rows = [] # will be a list for list for all rows
for row_num in range(len(body_rows)): # A row at a time
    row = [] # this will old entries for one row
    for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
        # row_item.text removes the tags from the entries
        # the following regex is to remove \xa0 and \n and comma from row_item.text
        # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        #append aa to row - note one row entry is being appended
        row.append(aa)
    # append one row to all_rows
    all_rows.append(row)
df = pd.DataFrame(data=all_rows,columns=['data','event'])
df.head()

['\nDecember 2019']


,data,event
0,12-31-19,Investigation Into Fatal Shooting At 9601 West...
1,12-31-19,Investigation Into Two Women Found Deceased At...
2,12-31-19,Investigation Into Fatal Crash At 6100 West Li...
3,12-30-19,Investigation Into Fatal Shooting At 5021 Long...
4,12-30-19,Investigation Into Fatal Shooting At 1000 Cros...


In [497]:
df.shape

(102, 2)

In [498]:
# Filter out other types of events cause we only focus on fatal shooting and crash.
df = df[df['event'].str.contains("Fatal")]
df.shape

(58, 2)

In [499]:
df['address'] = np.nan
for i in range(len(df)):
    df.iloc[i,2] = ''.join(re.findall('(?<=At ).*$',df.iloc[i,1])) + ", Houston, TX, USA" 
df.head()

,data,event,address
0,12-31-19,Investigation Into Fatal Shooting At 9601 West...,"9601 West Montgomery Road, Houston, TX, USA"
2,12-31-19,Investigation Into Fatal Crash At 6100 West Li...,"6100 West Little York Road, Houston, TX, USA"
3,12-30-19,Investigation Into Fatal Shooting At 5021 Long...,"5021 Longmeadow Street, Houston, TX, USA"
4,12-30-19,Investigation Into Fatal Shooting At 1000 Cros...,"1000 Crosby Street, Houston, TX, USA"
7,12-30-19,Investigation Into Fatal Crash At 8200 Telepho...,"8200 Telephone Road, Houston, TX, USA"


In [455]:
df['event_type'] = "Fatal Crash"
df.iloc[df['event'].str.contains("Shooting"),3] = "Fatal Shooting"
df.head()

,data,event,address,event_type
0,01-31-20,Investigation into Fatal Crash at 610 Sunnyside,"610 Sunnyside, Houston, TX, USA",Fatal Crash
1,01-31-20,Investigation into Fatal Shooting at 9202 Nath...,"9202 Nathaniel Street, Houston, TX, USA",Fatal Shooting
2,01-31-20,Investigation into Fatal Shooting at 800 South...,"800 South Wayside Drive, Houston, TX, USA",Fatal Shooting
3,01-31-20,Investigation into Fatal Shooting at 9002 Ster...,"9002 Sterlingshire Street, Houston, TX, USA",Fatal Shooting
5,01-31-20,Investigation into Fatal Shooting at 3600 Yose...,"3600 Yosemite Street, Houston, TX, USA",Fatal Shooting


In [456]:
df.to_csv(r"C:\Users\Fate\Downloads\ML\HPD\Dec2019_address.csv",index=False)

In [18]:
import glob
import os
os.chdir(r"C:\Users\Fate\Downloads\ML\HPD")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames

['Apr2020_address.csv',
 'Apr2021_address.csv',
 'Apr2022_address.csv',
 'Aug2020_address.csv',
 'Aug2021_address.csv',
 'Aug2022_address.csv',
 'Dec2020_address.csv',
 'Dec2021_address.csv',
 'Feb2020_address.csv',
 'Feb2021_address.csv',
 'Feb2022_address.csv',
 'Jan2020_address.csv',
 'Jan2021_address.csv',
 'Jan2022_address.csv',
 'Jul2020_address.csv',
 'Jul2021_address.csv',
 'Jul2022_address.csv',
 'Jun2020_address.csv',
 'Jun2021_address.csv',
 'Jun2022_address.csv',
 'Mar2020_address.csv',
 'Mar2021_address.csv',
 'Mar2022_address.csv',
 'May2020_address.csv',
 'May2021_address.csv',
 'May2022_address.csv',
 'Nov2020_address.csv',
 'Nov2021_address.csv',
 'Oct2020_address.csv',
 'Oct2021_address.csv',
 'Oct2022_address.csv',
 'Sep2020_address.csv',
 'Sep2021_address.csv',
 'Sep2022_address.csv']

In [19]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined.csv", index=False, encoding='utf-8-sig')

In [22]:
df = pd.read_csv(r"C:\Users\Fate\Downloads\ML\HPD\combined.csv")
df = df.iloc[:,1:]
df.head(30)

,data,event,address,event_type
0,04-30-20,Investigation into Fatal Crash at 5800 Eastex ...,"5800 Eastex Freeway, Houston, TX, USA",Fatal Crash
1,04-28-20,Investigation into Fatal Crash at 4300 Telepho...,"4300 Telephone Road, Houston, TX, USA",Fatal Crash
2,04-28-20,Investigation into Fatal Shooting at 3826 Seab...,"3826 Seabrook Street, Houston, TX, USA",Fatal Shooting
3,04-27-20,Investigation into Fatal Shooting at 8301 Darl...,"8301 Darlington Drive, Houston, TX, USA",Fatal Shooting
4,04-27-20,Investigation into Fatal Crash at 11100 Almeda...,"11100 Almeda Road, Houston, TX, USA",Fatal Crash
5,04-27-20,Investigation into Fatal Shooting at 10615 Mea...,"10615 Meadowglen Lane, Houston, TX, USA",Fatal Shooting
6,04-24-20,Investigation into Fatal Shooting at 4600 Whit...,"4600 White Rock Street, Houston, TX, USA",Fatal Shooting
7,04-24-20,Investigation into Fatal Shooting at 4714 Ward...,"4714 Ward Street, Houston, TX, USA",Fatal Shooting
8,04-24-20,Investigation into Fatal Shooting at 8900 Sout...,"8900 South Braeswood Boulevard, Houston, TX, USA",Fatal Shooting
9,04-23-20,Investigation into Fatal Crash at 6000 Nunn St...,"6000 Nunn Street, Houston, TX, USA",Fatal Crash


In [23]:
df.shape

(1513, 4)

In [2]:
import geopy
import geopandas
from geopy.geocoders import GoogleV3,Nominatim
locator = Nominatim(user_agent="myGeocoder")

geolocator = GoogleV3(api_key = "Your_API_Key_here")

In [25]:
location = geolocator.geocode("My office address, Houston, USA")
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

Latitude = 29.544439, Longitude = -95.5702195


In [26]:
latitude=[]
longitude=[]
for address in df['address']:
    latitude.append(geolocator.geocode(address).latitude)
    longitude.append(geolocator.geocode(address).longitude)
print(len(latitude))
df['latitude'] = latitude
df['longitude'] = longitude
df.head()

200


In [48]:
df.head()

,data,event,address,event_type,latitude,longitude
0,04-30-20,Investigation into Fatal Crash at 5800 Eastex ...,"5800 Eastex Freeway, Houston, TX, USA",Fatal Crash,29.812368,-95.331982
1,04-28-20,Investigation into Fatal Crash at 4300 Telepho...,"4300 Telephone Road, Houston, TX, USA",Fatal Crash,29.702930,-95.304678
2,04-28-20,Investigation into Fatal Shooting at 3826 Seab...,"3826 Seabrook Street, Houston, TX, USA",Fatal Shooting,29.684594,-95.366678
3,04-27-20,Investigation into Fatal Shooting at 8301 Darl...,"8301 Darlington Drive, Houston, TX, USA",Fatal Shooting,29.834904,-95.275063
4,04-27-20,Investigation into Fatal Crash at 11100 Almeda...,"11100 Almeda Road, Houston, TX, USA",Fatal Crash,29.662690,-95.400054


In [14]:
df['road_name'] = np.nan
df.head()

,Unnamed: 0,data,event,address,event_type,latitude,longitude,road,road_name
0,0,04-30-20,Investigation into Fatal Crash at 5800 Eastex ...,"5800 Eastex Freeway, Houston, TX, USA",Fatal Crash,29.812368,-95.331982,5800 Eastex Freeway,NaN
1,1,04-28-20,Investigation into Fatal Crash at 4300 Telepho...,"4300 Telephone Road, Houston, TX, USA",Fatal Crash,29.702930,-95.304678,4300 Telephone Road,NaN
2,2,04-28-20,Investigation into Fatal Shooting at 3826 Seab...,"3826 Seabrook Street, Houston, TX, USA",Fatal Shooting,29.684594,-95.366678,3826 Seabrook Street,NaN
3,3,04-27-20,Investigation into Fatal Shooting at 8301 Darl...,"8301 Darlington Drive, Houston, TX, USA",Fatal Shooting,29.834904,-95.275063,8301 Darlington Drive,NaN
4,4,04-27-20,Investigation into Fatal Crash at 11100 Almeda...,"11100 Almeda Road, Houston, TX, USA",Fatal Crash,29.662690,-95.400054,11100 Almeda Road,NaN


In [16]:
for i in range(len(dfAll)):
    df.iloc[i,8] = ' '.join(df.iloc[i,7].split(" ")[1:])
df.head()

,Unnamed: 0,data,event,address,event_type,latitude,longitude,road,road_name
0,0,04-30-20,Investigation into Fatal Crash at 5800 Eastex ...,"5800 Eastex Freeway, Houston, TX, USA",Fatal Crash,29.812368,-95.331982,5800 Eastex Freeway,Eastex Freeway
1,1,04-28-20,Investigation into Fatal Crash at 4300 Telepho...,"4300 Telephone Road, Houston, TX, USA",Fatal Crash,29.702930,-95.304678,4300 Telephone Road,Telephone Road
2,2,04-28-20,Investigation into Fatal Shooting at 3826 Seab...,"3826 Seabrook Street, Houston, TX, USA",Fatal Shooting,29.684594,-95.366678,3826 Seabrook Street,Seabrook Street
3,3,04-27-20,Investigation into Fatal Shooting at 8301 Darl...,"8301 Darlington Drive, Houston, TX, USA",Fatal Shooting,29.834904,-95.275063,8301 Darlington Drive,Darlington Drive
4,4,04-27-20,Investigation into Fatal Crash at 11100 Almeda...,"11100 Almeda Road, Houston, TX, USA",Fatal Crash,29.662690,-95.400054,11100 Almeda Road,Almeda Road


In [9]:
from geopy.geocoders import Nominatim, GoogleV3
  
# initialize Nominatim API 
locator = Nominatim(user_agent="geoapiExercises", timeout=None)
geolocator = GoogleV3(api_key = "Your_API_Key_here")
  
# place input by geek
place = "5500 North Freeway, Houston, TX, USA"
location = locator.geocode(place)
  
# traverse the data
data = location.raw
# print(str(geolocator.geocode(place)).split(",")[2].split(" ")[2])
loc_data = data['display_name'].split()
print("Full Location")
print(loc_data)
print("Zip code : ",loc_data[-3])

Full Location
['North', 'Freeway,', 'Houston,', 'Harris', 'County,', 'Texas,', '77022,', 'United', 'States']
Zip code :  77022,


In [32]:
zip = []
for i in df['address']:
    if locator.geocode(i) == None:
        zip.append("00000")
    else:
        zip.append(locator.geocode(i).raw['display_name'].split()[-3][:5])
df['zip'] = zip

200

In [34]:
df.head()

,data,event,address,event_type,latitude,longitude,road,road_name,zip
0,04-30-20,Investigation into Fatal Crash at 5800 Eastex ...,"5800 Eastex Freeway, Houston, TX, USA",Fatal Crash,29.812368,-95.331982,5800 Eastex Freeway,Eastex Freeway,77026
1,04-28-20,Investigation into Fatal Crash at 4300 Telepho...,"4300 Telephone Road, Houston, TX, USA",Fatal Crash,29.702930,-95.304678,4300 Telephone Road,Telephone Road,77087
2,04-28-20,Investigation into Fatal Shooting at 3826 Seab...,"3826 Seabrook Street, Houston, TX, USA",Fatal Shooting,29.684594,-95.366678,3826 Seabrook Street,Seabrook Street,77021
3,04-27-20,Investigation into Fatal Shooting at 8301 Darl...,"8301 Darlington Drive, Houston, TX, USA",Fatal Shooting,29.834904,-95.275063,8301 Darlington Drive,Darlington Drive,77028
4,04-27-20,Investigation into Fatal Crash at 11100 Almeda...,"11100 Almeda Road, Houston, TX, USA",Fatal Crash,29.662690,-95.400054,11100 Almeda Road,Almeda Road,77021


In [35]:
df.to_csv(r"C:\Users\Fate\Downloads\ML\HPD\df3years.csv",index=False)